In [30]:
from web3 import Web3
import pandas as pd
import requests

In [3]:
web3 = Web3(Web3.HTTPProvider("https://intensive-sly-mountain.quiknode.pro/a3f5256d7f2af6541d483cce3f1d49c94c01879e/"))
print("\033[92m"+str(web3.is_connected()))

True


Figure out how to get cost & profit from 2 transactions

In [6]:
def get_internal_transactions(tx_hash):
    # API endpoint
    url = 'https://api.etherscan.io/api'

    # Parameters
    params = {
        'module': 'account',
        'action': 'txlistinternal',
        'txhash': tx_hash,
        'apikey': '1PN1111XBM2W5HIQCSMQH6RA65JVYPQM1R'
    }

    try:
        # Sending GET request
        response = requests.get(url, params=params, timeout=3)
    
        # Checking if request was successful
        if response.status_code == 200:
            data = response.json()
            return data["result"]
        else:
            #print('Error occurred:', response.status_code)
            return None
        
    except requests.exceptions.Timeout:
        #print('Request did not go through: timeout occurred')
        return None

In [80]:
def get_amount_eth(transaction_hash):
    
    internal_transaction = get_internal_transactions(transaction_hash)
    amount_in_wei = int(internal_transaction[0]["value"])
    return amount_in_wei / 10**18

Example 1

In [45]:
attack_1 = "0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090" 
victim = "0xb46caa3c254f3e2050818a1ce3ea6b01f96011b403253c620c9a1720c4a6612a" 
attack_2 = "0x9b4d25a15945cc913732642dc2f9d7b655eac4d8be624f8098c102d653d9c969"

In [24]:
get_amount_eth(attack_1)

6.180859375

In [25]:
get_amount_eth(attack_2)

6.337101476298321

Example 2

In [26]:
attack_1 = "0x514065a6804e3012f23308c4ea203d16c53d6d36dad1b54438f0f2b9e45b082b" 
victim = "0xba4a605237e59f282f8b662a9ab024c16aac017bd97a0839faf1a8f6bea24638" 
attack_2 = "0x87329e5bf34a9262694287fb62f2389d704f056b588a46e4439df7d03b47fbdc"


In [27]:
get_amount_eth(attack_1)


0.7845739618270202

In [29]:
get_amount_eth(attack_2)


0.795890884390742

Get gas fees spent

In [75]:
def get_amount_gas_spent(transaction_hash):
    
    transaction = web3.eth.get_transaction(transaction_hash)
    gas_used = transaction["gas"] 
    gas_price = transaction["gasPrice"] / 10**18

    return gas_used * gas_price

In [97]:
def get_amount_gas_spent(transaction_hash):
    
    transaction = web3.eth.get_transaction_receipt(transaction_hash)
    gas_used = transaction["gasUsed"] 
    gas_price = transaction["effectiveGasPrice"] / 10**18

    return gas_used * gas_price

In [98]:
print("\033[92m"+str(get_amount_gas_spent(attack_1)))

0.00303018


In [99]:
print("\033[92m"+str(get_amount_gas_spent(attack_2)))

0.002185655


Cost: amount of eth spent for A1 and fees from A1 and A2  
Profit: amount of eth received for A2 - cost

In [100]:
def calculate_cost(attack_1, attack_2):
    
    eth_spent = get_amount_eth(attack_1)
    eth_fees = get_amount_gas_spent(attack_1) + get_amount_gas_spent(attack_2)
    
    return eth_spent + eth_fees

In [101]:
calculate_cost(attack_1, attack_2)

6.186075209999999

In [105]:
def calculate_profit(cost, attack_2):
    
    eth_received = get_amount_eth(attack_2)
    
    return eth_received - cost

In [124]:
cost = calculate_cost(attack_1, attack_2)
profit = calculate_profit(cost, attack_2)

In [121]:
def get_eth_rate_at_time(timestamp):
    # CryptoCompare API endpoint for historical Ethereum price
    url = f"https://min-api.cryptocompare.com/data/pricehistorical?fsym=ETH&tsyms=USD&ts={timestamp}&extraParams=your_app_name"
    
    # Send GET request to CryptoCompare API
    response = requests.get(url)
    
    # Parse JSON response
    data = response.json()
    
    # Check if response contains data
    if 'ETH' in data and 'USD' in data['ETH']:
        eth_price_usd = data['ETH']['USD']
        return eth_price_usd
    else:
        return None

In [113]:
def get_block_timestamp(block_number):
    block = web3.eth.get_block(block_number)
    timestamp = block['timestamp']
    return timestamp

In [143]:
timestamp = get_block_timestamp(5574870)
eth_rate_at_timestamp = get_eth_rate_at_time(timestamp)
eth_rate_at_timestamp

747.79

In [125]:
profit * eth_rate_at_timestamp

112.93593167522216

In [126]:
cost * eth_rate_at_timestamp

4625.885181285899

In [128]:
(get_amount_gas_spent(attack_1) + get_amount_gas_spent(attack_2)) * eth_rate_at_timestamp

3.90034925465